---

📌 **This notebook has been updated in [jhj0517/finetuning-notebooks](https://github.com/jhj0517/finetuning-notebooks) repository!**

## Version : 1.0.0
---

In [1]:
#@title #(Optional) Check GPU

#@markdown To train Hunyuan Video lora 24GB VRAM is recommended.
#@markdown  <br>If your dataset contains videos, then more than 24GB is recommended.
#@markdown <br>You can check your GPU setup before start.
!nvidia-smi

Tue Feb 25 23:03:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             41W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
#@title #1. Install Dependencies
#@markdown This notebook is powered by https://github.com/tdrussell/diffusion-pipe
!git clone --recurse-submodules https://github.com/tdrussell/diffusion-pipe
%cd diffusion-pipe

# Cherry picked dependencies to use in Colab.
!pip install deepspeed
!pip install datasets
!pip install torch-optimi
!pip install bitsandbytes
!pip install av
!pip install loguru
!pip install flash-attn


# Comment on the requirements above, and uncomment below if you're not using Colab.
# !pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
# !pip install deepspeed
# !pip install toml
# !pip install transformers
# !pip install diffusers>=0.32.1
# !pip install datasets
# !pip install pillow
# !pip install sentencepiece
# !pip install protobuf
# !pip install peft
# !pip install torch-optimi
# !pip install tensorboard
# !pip install tqdm
# !pip install safetensors
# !pip install bitsandbytes
# !pip install imageio[ffmpeg]
# !pip install av
# !pip install einops
# !pip install accelerate
# !pip install loguru
# !pip install flash-attn; sys_platform==linux
# !pip install omegaconf
# !pip install iopath
# !pip install termcolor
# !pip install hydra-core

Cloning into 'diffusion-pipe'...
remote: Enumerating objects: 715, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 715 (delta 243), reused 235 (delta 217), pack-reused 443 (from 1)
Receiving objects: 100% (715/715), 7.80 MiB | 19.06 MiB/s, done.
Resolving deltas: 100% (444/444), done.
Submodule 'submodules/Cosmos' (https://github.com/NVIDIA/Cosmos) registered for path 'submodules/Cosmos'
Submodule 'submodules/HunyuanVideo' (https://github.com/Tencent/HunyuanVideo) registered for path 'submodules/HunyuanVideo'
Submodule 'submodules/Lumina_2' (https://github.com/Alpha-VLLM/Lumina-Image-2.0) registered for path 'submodules/Lumina_2'
Cloning into '/content/diffusion-pipe/submodules/Cosmos'...
remote: Enumerating objects: 392, done.        
remote: Counting objects: 100% (115/115), done.        
remote: Compressing objects: 100% (81/81), done.        
remote: Total 392 (delta 63), reused 34 (delta 34), pack-reused 277 (fro

In [5]:
#@title # 2. (Optional) Mount Google Drive

#@markdown It's not mandatory but it's recommended to mount to Google Drive and use the Google Drive's path for your training dataset.

#@markdown When training Hunyuan Lora, the dataset could contatin both images and videos.

#@markdown Each file should have a corresponding text file (`.txt`) with the same name. <br>
#@markdown **Each video must have a specific number of frames, as much as you will define later in "frame_buckets".**

#@markdown The text file contains prompts associated with the video or image.


#@markdown ### Example Dataset Structure:
#@markdown ```
#@markdown your-dataset/
#@markdown ├── a (1).mp4         # Video file
#@markdown ├── a (1).txt         # Corresponding prompt for a (1).mp4
#@markdown ├── a (2).mp4         # Another video file
#@markdown ├── a (2).txt         # Corresponding prompt for a (2).mp4
#@markdown ├── a (3).png         # Image file
#@markdown ├── a (3).txt         # Corresponding prompt for a (3).png
#@markdown ```

from google.colab import drive
import os
drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

In [11]:
WAN_VIDEO_URL = {
    "Wan2_1-T2V-1_3B_bf16.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors",
    "Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors",
    "Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors",
}
url = WAN_VIDEO_URL["Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors"]
BASE_MODELS_DIR_PATH = "/content/drive/MyDrive/finetuning-notebooks/wan/base_models" # @param {type:"string"}

!wget {url} -P {BASE_MODELS_DIR_PATH}

--2025-02-25 23:37:34--  https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.88, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/6a/f7/6af72071ed19a24fd36436ce0baf194019423c9eaa7c8b82f2fa5f3ac60c31b6/996dbad030df09b0b3c8e764f0fb5a81b98b220ab89524d6a9369e9ed882791f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors%3B+filename%3D%22Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors%22%3B&Expires=1740530254&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDUzMDI1NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzZhL2Y3LzZhZjcyMDcxZWQxOWEyNGZkMzY0MzZjZTBiYWYxOTQwMTk0MjNjOWVhYTdjOGI4MmYyZmE1ZjNhYzYwYzMxYjYvOTk2ZGJhZDAzMGRmMDliMGI

In [7]:
#@title # 3. (Optional) Register Huggingface Token To Download Base Model

#@markdown This cell will download base models. If you don't already have the base model files in your google drive, run this.

#@markdown You need Huggingface token (Read permission) to run this.

#@markdown Get your tokens from https://huggingface.co/settings/tokens, and register in colab's seceret as **`HF_TOKEN`** and use it in any notebook. ( 'Read' permission is enough )

#@markdown To register secrets in colab, click on the key-shaped icon in the left panel and enter your **`HF_TOKEN`** like this:

#@markdown ![image](https://media.githubusercontent.com/media/jhj0517/finetuning-notebooks/master/docs/screenshots/colab_secrets.png)

import huggingface_hub
import os

# Set params
BASE_MODELS_DIR_PATH = "/content/drive/MyDrive/finetuning-notebooks/wan/base_models" # @param {type:"string"}
WAN_VIDEO = "Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors" #@param ["Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors", "Wan2_1-T2V-1_3B_bf16.safetensors", "Wan2_1-T2V-14B_fp8_e4m3fn.safetensors", "Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors", "Wan2_1-I2V-14B-720P_fp8_e4m3fn.safetensors"]
VAE = "Wan2_1_VAE_bf16.safetensors" #@param ["Wan2_1_VAE_bf16.safetensors", "Wan2_1_VAE_fp32.safetensors"]
CLIP = "open-clip-xlm-roberta-large-vit-huge-14_fp16.safetensors" #@param ["open-clip-xlm-roberta-large-vit-huge-14_fp16.safetensors"]
LLM = "umt5-xxl-enc-bf16.safetensors" #@param ["umt5-xxl-enc-bf16.safetensors"]

#@markdown Models will be downloaded from
#@markdown - https://huggingface.co/Kijai/WanVideo_comfy

WAN_VIDEO_URL = {
    "Wan2_1-T2V-1_3B_bf16.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors",
    "Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors",
    "Wan2_1-T2V-14B_fp8_e4m3fn.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-14B_fp8_e4m3fn.safetensors",
    "Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-I2V-14B-480P_fp8_e4m3fn.safetensors",
    "Wan2_1-I2V-14B-720P_fp8_e4m3fn.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-I2V-14B-720P_fp8_e4m3fn.safetensors",
}

WAN_VIDEO_VAE_URL = {
    "Wan2_1_VAE_bf16.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1_VAE_bf16.safetensors",
    "Wan2_1_VAE_fp32.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1_VAE_fp32.safetensors",
}

CLIP_URL = {
    "open-clip-xlm-roberta-large-vit-huge-14_fp16.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/open-clip-xlm-roberta-large-vit-huge-14_fp16.safetensors"
}

LLM_URL = {
    "umt5-xxl-enc-bf16.safetensors": "https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/umt5-xxl-enc-bf16.safetensors"
}

# Download models
video_url, vae_url = WAN_VIDEO_URL[WAN_VIDEO], WAN_VIDEO_VAE_URL[VAE]
clip_url, llm_url = CLIP_URL[CLIP], LLM_URL[LLM]

!wget {video_url} -P {BASE_MODELS_DIR_PATH}
!wget {vae_url} -P {BASE_MODELS_DIR_PATH}
!wget {clip_url} -P {BASE_MODELS_DIR_PATH}
!wget {llm_url} -P {BASE_MODELS_DIR_PATH}



--2025-02-25 23:21:24--  https://huggingface.co/Kijai/WanVideo_comfy/resolve/main/Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/6a/f7/6af72071ed19a24fd36436ce0baf194019423c9eaa7c8b82f2fa5f3ac60c31b6/a913893ce03cd6e3d48853258f4521f054db6d72b59d67359ea0b138d8071812?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors%3B+filename%3D%22Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors%22%3B&Expires=1740529284&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MDUyOTI4NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzZhL2Y3LzZhZjcyMDcxZWQxOWEyNGZkMzY0MzZjZTBiYWYxOTQwMTk0MjNjOWVhYTdjOGI4MmYyZmE1ZjNhYzYwYzMxYjYvYTkxMzg5M2NlMDNjZDZlM2Q0ODg1MzI1OGY0N

In [12]:
#@title # 4. Train with Parameters
import os
import sys
import toml

#@markdown If you're intended to train Lora from previous checkpoint, check this.
RESUME_FROM_CHECKPOINT = False #@param {type:"boolean"}

#@markdown ## Paths Configuration
OUTPUT_LORA_NAME = "My-Wan-Lora-V1" # @param {type:"string"}
OUTPUT_LORA_DIR_PATH = "/content/drive/MyDrive/finetuning-notebooks/wan/outputs"  # @param {type:"string"}
BASE_WAN_VIDEO_PATH = "/content/drive/MyDrive/finetuning-notebooks/wan/base_models/wan/Wan2_1-T2V-1_3B_fp8_e4m3fn.safetensors" # @param {type:"string"}
BASE_VAE_PATH = "/content/drive/MyDrive/finetuning-notebooks/wan/base_models/wan/Wan2_1_VAE_bf16.safetensors" # @param {type:"string"}
BASE_CLIP_PATH = "/content/drive/MyDrive/finetuning-notebooks/wan/base_models/wan/open-clip-xlm-roberta-large-vit-huge-14_fp16.safetensors" # @param {type:"string"}
BASE_LLM_PATH = "/content/drive/MyDrive/finetuning-notebooks/wan/base_models/wan/umt5-xxl-enc-bf16.safetensors" # @param {type:"string"}
DATASET_PATH = "/content/drive/MyDrive/finetuning-notebooks/dataset/dog-prompt" # @param {type:"string"}

OUTPUT_LORA_DIR_PATH = os.path.join(OUTPUT_LORA_DIR_PATH, OUTPUT_LORA_NAME)

#@markdown ## Dataset Configuration
#@markdown - **`frame_buckets`** is the list of frame numbers in your dataset.
#@markdown <br>For example, if your dataset contains 30, 60 frames of videos, then use : [30, 60]
#@markdown <br>Don't use too long frames unless you don't have a lot of VRAM.
#@markdown <br>If your dataset also contains images, then use : [1, 30, 60]
#@markdown - **`resolutions`** is the list of resolutions which **`diffusion-pipe`** will resize your dataset.
#@markdown <br>**`diffusion-pipe`** is smart to handle resizing your dataset by 1:2 or 2:1 image etc.
#@markdown <br>If you have less than 24GB of VRAM, just set it to 512, then increase it according to your device.
## Frame Buckets Settings
frame_buckets = [1]  # @param {type:"raw"}
# You can use 1024 if you have 24 GB > VRAM.
resolutions = [512]  # @param {type:"raw"}
## Aspect Ratio Bucketing Settings
enable_ar_bucket = True  # @param {type:"boolean"}
min_ar = 0.5  # @param {type:"number"}
max_ar = 2.0  # @param {type:"number"}
num_ar_buckets = 7  # @param {type:"integer"}
# Reduce as necessary
num_repeats = 5

# Write dataset.toml
dataset_config = {
    "resolutions": resolutions,
    "frame_buckets": frame_buckets,

    "enable_ar_bucket": enable_ar_bucket,
    "min_ar": min_ar,
    "max_ar": max_ar,
    "num_ar_buckets": num_ar_buckets,
    "directory": [
        {
            "path": DATASET_PATH,
            "num_repeats": num_repeats,
        }
    ],
}

os.makedirs(OUTPUT_LORA_DIR_PATH, exist_ok=True)
os.makedirs(DATASET_PATH, exist_ok=True)
dataset_config_file_path = os.path.join(DATASET_PATH, "dataset.toml")
with open(dataset_config_file_path, "w") as toml_file:
    toml.dump(dataset_config, toml_file)
print(f"dataset.toml is saved to {dataset_config_file_path}")

#@markdown ## Base Model Configuration

model_type = 'hunyuan-video'
dtype = 'bfloat16'  # @param {type:"string"}
transformer_dtype = 'float8'  # @param {type:"string"}
timestep_sample_method = 'logit_normal'  # @param {type:"string"}

#@markdown ## Training Settings
epochs = 50  # @param {type:"integer"}
# Batch size of a single forward/backward pass for one GPU.
micro_batch_size_per_gpu = 1  # @param {type:"integer"}
# Pipeline parallelism degree. A single instance of the model is divided across this many GPUs.
pipeline_stages = 1  # @param {type:"integer"}
gradient_accumulation_steps = 4  # @param {type:"integer"}
gradient_clipping = 1.0  # @param {type:"number"}
# Learning rate warmup.
warmup_steps = 50  # @param {type:"integer"}

#@markdown ## Eval Settings
eval_every_n_epochs = 1  # @param {type:"integer"}
eval_before_first_step = True  # @param {type:"boolean"}
eval_micro_batch_size_per_gpu = 1  # @param {type:"integer"}
eval_gradient_accumulation_steps = 1  # @param {type:"integer"}

#@markdown ## Lora Settings
adapter_type = 'lora'
rank = 32  # @param {type:"integer"}
adapter_dtype = 'bfloat16'  # @param {type:"string"}

# Optimizer settings
optimizer_type = 'adamw_optimi'  # @param {type:"string"}
lr = 2e-5  # @param {type:"number"}
betas = [0.9, 0.99]  # @param {type:"raw"}
weight_decay = 0.02  # @param {type:"number"}
eps = 1e-8  # @param {type:"number"}

#@markdown ## Misc Settings
save_every_n_epochs = 10  # @param {type:"integer"}
checkpoint_every_n_minutes = 30  # @param {type:"integer"}
activation_checkpointing = True  # @param {type:"boolean"}
partition_method = 'parameters'  # @param {type:"string"}
save_dtype = 'bfloat16'  # @param {type:"string"}
caching_batch_size = 1  # @param {type:"integer"}
steps_per_print = 1  # @param {type:"integer"}
video_clip_mode = 'single_middle'  # @param {type:"string"}


# Write config.toml
train_config = {
    "output_dir": OUTPUT_LORA_DIR_PATH,
    "dataset": dataset_config_file_path,

    # Training Settings
    "epochs": epochs,
    "micro_batch_size_per_gpu": micro_batch_size_per_gpu,
    "pipeline_stages": pipeline_stages,
    "gradient_accumulation_steps": gradient_accumulation_steps,
    "gradient_clipping": gradient_clipping,
    "warmup_steps": warmup_steps,

    # Eval Settings
    "eval_every_n_epochs": eval_every_n_epochs,
    "eval_before_first_step": eval_before_first_step,
    "eval_micro_batch_size_per_gpu": eval_micro_batch_size_per_gpu,
    "eval_gradient_accumulation_steps": eval_gradient_accumulation_steps,

    # Misc Settings
    "save_every_n_epochs": save_every_n_epochs,
    "checkpoint_every_n_minutes": checkpoint_every_n_minutes,
    "activation_checkpointing": activation_checkpointing,
    "partition_method": partition_method,
    "save_dtype": save_dtype,
    "caching_batch_size": caching_batch_size,
    "steps_per_print": steps_per_print,
    "video_clip_mode": video_clip_mode,

    "model": {
        "type": model_type,
        "transformer_path": BASE_WAN_VIDEO_PATH,
        "vae_path": BASE_VAE_PATH,
        "llm_path": BASE_LLM_PATH,
        "clip_path": BASE_CLIP_PATH,
        "dtype": dtype,
        "transformer_dtype": transformer_dtype,
        "timestep_sample_method": timestep_sample_method,
    },

    "adapter": {
        "type": "lora",
        "rank": rank,
        "dtype": adapter_dtype,
    },

    "optimizer": {
        "type": optimizer_type,
        "lr": lr,
        "betas": betas,
        "weight_decay": weight_decay,
        "eps": eps,
    },


}

train_config_file_path = os.path.join(DATASET_PATH, "config.toml")
with open(train_config_file_path, "w") as toml_file:
    toml.dump(train_config, toml_file)
print(f"config.toml is saved to {train_config_file_path}")


## Train
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

if RESUME_FROM_CHECKPOINT:
  !deepspeed --num_gpus=1 train.py --deepspeed --config {train_config_file_path} --resume_from_checkpoint
else:
  !deepspeed --num_gpus=1 train.py --deepspeed --config {train_config_file_path}


dataset.toml is saved to /content/drive/MyDrive/finetuning-notebooks/dataset/dog-prompt/dataset.toml
config.toml is saved to /content/drive/MyDrive/finetuning-notebooks/dataset/dog-prompt/config.toml
[2025-02-25 23:46:02,657] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-02-25 23:46:11.946150: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-25 23:46:11.963104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740527171.985396   11780 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E